<a href="https://colab.research.google.com/github/vincent4u/CE807_Text_Analytics/blob/main/Assignment_code12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Student ID: 2314555
Vincent Nnamdi UGWAH



Installing all required libraries.

In [30]:
import numpy as np
import os
import pickle
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re
from tqdm import tqdm

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**student id input as a variable for seeding**

In [31]:
student_id = 2314555  #my student reg number

Let's set `seed` for all libraries like `torch`, `numpy` etc as my student id

In [32]:
# setting studen_id as same seeds for all libraries

#numpy seed
np.random.seed(student_id)

# Common Codes

This section includes all common codes, for examples


*   Data read
*   Command Line argument reading
*   Performance Matrics
*   Print Dataset Statistics
*   Saving model and output
*   Loading Model and output
*   etc




**Setting first the GDrive to access and set data and model paths**

For examples,

student_id = 12345670

set GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = ‘./CE807-24-SP/Lab10/’ in your GDrive

now set all global variable,


Sample output directory and file structure: https://drive.google.com/drive/folders/1okgSzgGiwPYYFp7NScEt9MNVolOlld1d?usp=share_link   

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [33]:
# Initializing GDrive and data and models paths

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = './CE807-24-SP/Assignment/'
GOOGLE_DRIVE_PATH = os.path.join('gdrive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print('List files: ', os.listdir(GOOGLE_DRIVE_PATH))

DATA_PATH = os.path.join(GOOGLE_DRIVE_PATH, 'data', '5') # Directing path to data type 5, since my student numbers ends with 5
train_file = os.path.join(DATA_PATH, 'train.csv')         # Reading the Train CSV file
print('Train file: ', train_file)

val_file = os.path.join(DATA_PATH, 'valid.csv')           # Reading the Validation CSV file
print('Validation file: ', val_file)

test_file = os.path.join(DATA_PATH, 'test.csv')            # Reading the Test CSV file
print('Test file: ', test_file)


MODEL_PATH = os.path.join(GOOGLE_DRIVE_PATH, 'model', str(student_id)) # Making sure to use student Regitration number already called in the begining
MODEL_Gen_DIRECTORY = os.path.join(MODEL_PATH, 'Model_Gen')            # Creating Path to Model Generative directory
print('Model Generative directory: ', MODEL_Gen_DIRECTORY)

MODEL_Gen_File = MODEL_Gen_DIRECTORY + '.zip'


MODEL_Dis_DIRECTORY = os.path.join(MODEL_PATH, 'Model_Dis')           # Creating Path to Model Discriminative directory
print('Model Discriminative directory: ', MODEL_Dis_DIRECTORY)

MODEL_Dis_File = MODEL_Dis_DIRECTORY + '.zip'                         # Zipping the FIle


List files:  ['data', 'model']
Train file:  gdrive/MyDrive/./CE807-24-SP/Assignment/data/5/train.csv
Validation file:  gdrive/MyDrive/./CE807-24-SP/Assignment/data/5/valid.csv
Test file:  gdrive/MyDrive/./CE807-24-SP/Assignment/data/5/test.csv
Model Generative directory:  gdrive/MyDrive/./CE807-24-SP/Assignment/model/2314555/Model_Gen
Model Discriminative directory:  gdrive/MyDrive/./CE807-24-SP/Assignment/model/2314555/Model_Dis


Visualing the train file for possible steps to Preprocessing

In [34]:
train_df = pd.read_csv(train_file)
train_df.head()

,comment_id,comment,split,toxicity
0,579986655.0,SDATA_5 : NEWLINE_TOKENNEWLINE_TOKEN== Kinder...,train,0
1,86029332.0,SDATA_5 : NEWLINE_TOKENNEWLINE_TOKENDo not vi...,train,0
2,169864624.0,SDATA_5 : NEWLINE_TOKEN:::Judging by some of ...,train,0
3,96482757.0,SDATA_5 : I LOVE FATTY SEX MMMM : EDATA_5,train,0
4,327661902.0,"SDATA_5 : NEWLINE_TOKEN:Please, could you exp...",train,0


# ***Computing the Performance Metrics***
I am going to use different performance matrics like Accuracy, Recall (macro), Precision (macro), F1 (macro) and Confusion Matrix for the performance evaluation. This will print all the matrics and display Confusion Matrix with proper X & Y axis labels

In [35]:
#Writting the function for the computing the performance metrics

def compute_performance(y_true, y_pred):
    # Computing metrics
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    # Print metrics
    print('Accuracy:', accuracy)
    print('Recall (macro):', recall)
    print('Precision (macro):', precision)
    print('F1 Score (macro):', f1)

    # Compute and print confusion matrix
    labels = np.unique(np.concatenate((y_true, y_pred)))
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    print('Confusion Matrix:')
    print(cm)

    return f1

# ***Saving and Loading Model ***

In [36]:

def save_model(model,model_dir):
  # save the model to disk
  # Check if the Model directory exists

  if not os.path.exists(model_dir):
      # Creating the directory if it doesn't exist
      os.makedirs(model_dir)
      print(f"Directory '{model_dir}' created successfully.")
  else:
      print(f"Directory '{model_dir}' already exists.")

  model_file = os.path.join(model_dir, 'model.sav')
  pickle.dump(model, open(model_file, 'wb'))

  print('Saved model to ', model_file)

  return model_file

def load_model(model_file):
    # loading model from disk

    model = pickle.load(open(model_file, 'rb'))

    print('Loaded model from ', model_file)

    return model

# Downloading GDrive Link into a directory

In [37]:
import requests

def extract_file_id_from_url(url):
    # Extract the file ID from the URL
    file_id = None
    if 'drive.google.com' in url:
        file_id = url.split('/')[-2]
    elif 'https://docs.google.com' in url:
        file_id = url.split('/')[-1]

    return file_id

def download_file_from_drive(file_id, file_path):
    # Constructing the download URL
    download_url = f"https://drive.google.com/uc?id={file_id}"

    # Download the file
    response = requests.get(download_url)
    if response.status_code == 200:
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print("File downloaded successfully!",file_path)
    else:
        print("Failed to download the file.")

def download_zip_file_from_link(file_url,file_path):

  file_id = extract_file_id_from_url(file_url)
  if file_id:
      download_file_from_drive(file_id, file_path)
  else:
      print("Invalid Google Drive URL.")


# Zip and Unzip a GDrive File

In [38]:
import zipfile
import shutil
import os

# Function to zip a directory
def zip_directory(directory, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(directory, '..')))
        print('Created a zip file',zip_filename)

# Function to unzip a zip file
def unzip_file(zip_filename, extract_dir):
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print('Extracted a zip file to',extract_dir)

# Example usage:
# directory_to_zip = 'path/to/your/directory'
# zip_filename = 'output_zipfile.zip'

# # Zip the directory
# zip_directory(directory_to_zip, zip_filename)

# # Unzip the zip file
# extract_dir = 'path/to/extract'
# unzip_file(zip_filename, extract_dir)


# Get Sharable link of your Zip file in Gdrive

In [39]:
!pip install -U -q PyDrive

In [41]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


def get_gdrive_link(file_path):
    # Authenticating and creating PyDrive client
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    # Find the file in Google Drive
    file_name = file_path.split('/')[-1]
    file_list = drive.ListFile({'q': f"title='{file_name}'"}).GetList()

    # Get the file ID and generate the shareable link
    if file_list:
        file_id = file_list[0]['id']
        gdrive_link = f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"
        return gdrive_link
    else:
        return "File not found in Google Drive"

def get_shareable_link(url):

    file_id = extract_file_id_from_url(url)

    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    try:
        file_obj = drive.CreateFile({'id': file_id})
        file_obj.FetchMetadata()
        file_obj.InsertPermission({
            'type': 'anyone',
            'value': 'anyone',
            'role': 'reader'
        })

        # Getting the shareable link
        return file_obj['alternateLink']
    except Exception as e:
        print("Error:", e)
        return None


# ***Loading all Libraries***

In [42]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re
from tqdm import tqdm

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Method Generative Start

This section includes all the details for the Generative Model (Naive Bayes).
It includes the following:

*   Data reading
*   Data clearning, if any
*   Convert data to vector/tokenization/vectorization
*   Model Declaration/Initialization/building
*   Training and validation of the model using training and validation dataset
*   Save the trained model
*   Load and Test the model on testing set
*   Save the output of the model




## Training Generative Method Code


# ***Step 1: Data Cleaning and Preprocessing***

In [43]:
    # Define preprocessing function
    def preprocess_text(text):
        # Remove specific tokens
        text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")
        # Remove special characters and punctuation marks using regular expressions
        text = re.sub(r'[^\w\s]', '', text)
        # Remove URLs, email addresses, and phone numbers
        text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)
        # Remove numbers
        text = re.sub(r"\d+", "", text)
        # Remove non-ASCII characters
        text = text.encode("ascii", "ignore").decode()
        # Convert the text to lowercase
        text = text.lower()
        # Remove extra white spaces
        text = re.sub(r"\s+", " ", text)
        # Tokenize the text
        tokens = nltk.word_tokenize(text)
        # Remove stop words
        stop_words = set(stopwords.words("english"))
        tokens = [word for word in tokens if word not in stop_words]
        # Apply lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        # Join the tokens back into a single string
        preprocessed_text = " ".join(tokens)
        return preprocessed_text


**Model training, saving and generating link**

In [50]:
def train_Gen(train_file, val_file, model_dir):

    # reading the train and validation file
    train_df = pd.read_csv(train_file)
    val_df = pd.read_csv(val_file)

    tqdm.pandas(desc="Preprocessing train data")
    train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
    tqdm.pandas(desc="Preprocessing val data")
    val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

    # Initialize the vectorizer
    vectorizer = TfidfVectorizer()
    # Fit the vectorizer on the training data and transform the training data
    train_features = vectorizer.fit_transform(train_df['preprocessed_comment'])
    # Transform the validation data using the fitted vectorizer
    val_features = vectorizer.transform(val_df['preprocessed_comment'])

    # Define the Naive Bayes model with TF-IDF vectorizer as a pipeline
    pipeline = Pipeline([
        ('tfidf', vectorizer),
        ('naive_bayes', MultinomialNB())
    ])

    # Defining the hyperparamneters used
    best_params = {
        'naive_bayes__alpha': 0.0010454648682604534,
        'naive_bayes__fit_prior': False,
    }

    #initializing the pipeline
    pipeline.set_params(**best_params)
    pipeline.fit(train_df['preprocessed_comment'], train_df['toxicity'])          #Training the Model

    #Predicitng on the Validation set
    y_pred = pipeline.predict(val_df['preprocessed_comment'])

    #computing the performance useing the compute_performance function
    score= compute_performance(val_df['toxicity'], y_pred)

    # print('Accuracy:', accuracy)
    # print('Recall (macro):', recall)
    # print('Precision (macro):', precision)
    # print('F1 Score (macro):', f1)
    # print('Confusion Matrix:')
    # print(cm)
    print(score)

    #saving the model and
    model_gdrive_link = save_and_share_model(pipeline, model_dir)
    return model_gdrive_link

def save_and_share_model(model, model_dir):
   # Model is working fine, so save model
    save_model(model, model_dir)
    # Now Zip Model to share it
    zip_directory(model_dir, MODEL_Gen_File)
    model_gdrive_link = get_gdrive_link(MODEL_Gen_File)
    print(model_gdrive_link)
    get_shareable_link(model_gdrive_link)
    return model_gdrive_link

In [ ]:
#  best_params = {
#         'naive_bayes__alpha': 0.0010454648682604534,
#         'naive_bayes__fit_prior': False,
#     }
#model_gdrive_link = train_Gen(train_file, val_file, MODEL_Gen_DIRECTORY)

## Testing Method generative (Naive Bayes) Code


In [52]:
def test_Gen(test_file, MODEL_PATH,model_gdrive_link):
    print('\n Start by downloading model')
    # Frist Get model from the link
    # model_gdrive_link = get_gdrive_link(model_file)

    # These two are temporary directory and file
    test_model_file = MODEL_PATH+'/test.zip'
    test_model_path = MODEL_PATH+'/test/'

    # Now download and unzip the model file
    download_zip_file_from_link(model_gdrive_link,test_model_file)
    print('Model downloaded to', test_model_file)
    unzip_file(test_model_file, test_model_path)
    print('\n Model is downloaded to ',test_model_path)



    test_df=pd.read_csv(test_file)
    print('\n Data is loaded from', test_file)

    #getting the model file name and loading it
    test_model_file=os.path.join(test_model_path,'Model_Gen','model.sav')

    model=load_model(test_model_file)

    #Doing the prediction
    y_pred=model.predict(test_df['comment'])

    # Now save the model output in the same test file
    # Note the name of output column, this is for the discriminative model
    test_df['out_label_model_Gen'] = y_pred

    # Now save the model output in the same output file
    test_df.to_csv(test_file, index=False)
    print('\n Output is save in ', test_file)

    return test_file

## Method Generative End


# Method Discriminative Start

This section includes all details of your Method 2.

*   Data reading
*   Data clearning, if any
*   Convert data to vector/tokenization/vectorization
*   Model Declaration/Initialization/building
*   Training and validation of the model using training and validation dataset
*   Save the trained model
*   Load and Test the model on testing set
*   Save the output of the model




## Training Method Discriminative Code


Importing all Libraries

In [53]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from tqdm.auto import tqdm


import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Data Preprocessing and Cleaning

In [54]:
def preprocess_text(text):
        # Remove specific tokens
        text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")
        # Remove special characters and punctuation marks using regular expressions
        text = re.sub(r'[^\w\s]', '', text)
        # Remove URLs, email addresses, and phone numbers
        text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)
        # Remove numbers
        text = re.sub(r"\d+", "", text)
        # Remove non-ASCII characters
        text = text.encode("ascii", "ignore").decode()
        # Convert the text to lowercase
        text = text.lower()
        # Remove extra white spaces
        text = re.sub(r"\s+", " ", text)
        # Tokenize the text
        tokens = nltk.word_tokenize(text)
        # Remove stop words
        stop_words = set(stopwords.words("english"))
        tokens = [word for word in tokens if word not in stop_words]
        # Apply lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        # Join the tokens back into a single string
        preprocessed_text = " ".join(tokens)
        return preprocessed_text

# ***Training the Descriminative Model***

In [59]:
def train_dis(train_file, val_file, model_dir):
    # Getting the train and validation datasets
    train_df = pd.read_csv(train_file)
    val_df = pd.read_csv(val_file)

    # Preprocess the text
    tqdm.pandas(desc="Preprocessing train data")
    train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
    tqdm.pandas(desc="Preprocessing val data")
    val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)

    best_params = {
    "objective": "reg:squarederror",
    'n_estimators': 859,
    'learning_rate': 0.09069799936088908,
    'max_depth': 10,
    'subsample': 0.7415235437197126,
    'colsample_bytree': 0.5241592558794433,
    'min_child_weight': 2
                          }


    # Create an instance of TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer on the training data and transform the training data
    train_features = vectorizer.fit_transform(train_df['preprocessed_comment'])

    # Transform the validation data using the fitted vectorizer
    val_features = vectorizer.transform(val_df['preprocessed_comment'])

    # Create an instance of XGBClassifier with the best parameters
    model = XGBClassifier(**best_params, tree_method='auto')

    # Fit the model on the training features and labels
    model.fit(train_features, train_df['toxicity'])

    # Predict on the validation features
    val_pred = model.predict(val_features)

    # Compute performance metrics
    score = compute_performance(val_df['toxicity'], val_pred)

    # Print performance metrics
    # print('Accuracy:', accuracy)
    # print('Recall (macro):', recall)
    # print('Precision (macro):', precision)
    # print('F1 Score (macro):', f1)
    # print('Confusion Matrix:')
    # print(cm)
    print(score)

    # Model is working fine, so save model
    save_model(model, model_dir)

    # Now Zip Model to share it
    zip_directory(model_dir, MODEL_Dis_File)

    model_gdrive_link = get_gdrive_link(MODEL_Dis_File)

    print(model_gdrive_link)
    get_shareable_link(model_gdrive_link)

    return model_gdrive_link


In [ ]:
# model_gdrive_link = train_dis(train_file, val_file, MODEL_Dis_DIRECTORY)
# best_params = {
#     "objective": "reg:squarederror",
#     'n_estimators': 859,
#     'learning_rate': 0.09069799936088908,
#     'max_depth': 10,
#     'subsample': 0.7415235437197126,
#     'colsample_bytree': 0.5241592558794433,
#     'min_child_weight': 2
# }

# Call the train_dis function and pass the best_params dictionary
#model_gdrive_link = train_dis(train_file, val_file, MODEL_Dis_DIRECTORY)

## Testing Method Discriminative Code
  

In [65]:
def test_dis(test_file, MODEL_PATH,model_gdrive_link):

    print('\n Start by downloading model')
    # Frist Get model from the link
    # model_gdrive_link = get_gdrive_link(model_file)

    # These two are temporary directory and file
    test_model_file = MODEL_PATH+'/test.zip'
    test_model_path = MODEL_PATH+'/test/'

    # Now download and unzip the model file
    download_zip_file_from_link(model_gdrive_link,test_model_file)
    print('Model downloaded to', test_model_file)
    unzip_file(test_model_file, test_model_path)
    print('\n Model is downloaded to ',test_model_path)



    test_df=pd.read_csv(test_file)
    print('\n Data is loaded from', test_file)

    #getting the model file name and loading it
    test_model_file=os.path.join(test_model_path,'Model_Dis','model.sav')

    model=load_model(test_model_file)

    #Doing the prediction
    y_pred=model.predict(test_df['comment'])

    # Now save the model output in the same test file
    # Note the name of output column, this is for the discriminative model
    test_df['out_label_model_Dis'] = y_pred

    # Now save the model output in the same output file
    test_df.to_csv(test_file, index=False)
    print('\n Output is save in ', test_file)

    return test_file

## Discriminative Method  End


# Other Method/model Start

In [61]:
import argparse

In [62]:
# Define argparse-like function
def parse_arguments(option):
    parser = argparse.ArgumentParser(description='Process some integers.')
    parser.add_argument('--option', '-o',  type=str, default=option, help='Description of your option.')
    args = parser.parse_args(args=[])
    return args

# Function to perform some action based on selected option
def perform_action(option):
    print("Performing action with option:", option)

    if option == '0':
      print('\n Okay Exiting!!! ')

    elif option == '1':
      print('\n Training Generative Model')
      model_gdrive_link = train_Gen(train_file,val_file,MODEL_Gen_DIRECTORY)
      print('Make sure to pass model URL in Testing',model_gdrive_link)

    elif option == '2':
      print('\n\n Pass the URL Not Variable !!!')
      print('\n Testing Generative Model')
      model_gen_url = ''
      test_Gen(test_file, model_gen_url)

    elif option == '3':
      print('\n Training Disciminative Model')
      model_gdrive_link = train_dis(train_file,val_file,MODEL_Dis_DIRECTORY)
      print('Make sure to pass model URL in Testing',model_gdrive_link)
      print('\n\n Pass the URL Not Variable !!!')

    elif option == '4':
      print('\n\n Pass the URL Not Variable !!!')
      print('\n Testing Disciminative Model')
      model_dis_url = 'https://drive.google.com/file/d/1--sJNJMLFvAz77HjiB8is8DDHOfY0FBG/view?usp=sharing'
      test_dis(test_file, MODEL_PATH, model_dis_url)

    else:
      print('Wrong Option Selected. \n\nPlease select Correct option')
      main()


def main():

    # Get option from user input
    user_option = input("0. To Exit Code\n"
                     "1. Train Model Generative\n"
                    "2. Test Model Generative\n"
                    "3. Train Model Discriminative\n"
                    "4. Test Model Discriminative\n"
                    "Enter your option: ")

    args = parse_arguments(user_option)
    option = args.option
    perform_action(option)


In [63]:
main()

0. To Exit Code
1. Train Model Generative
2. Test Model Generative
3. Train Model Discriminative
4. Test Model Discriminative
Enter your option: 1
Performing action with option: 1

 Training Generative Model


Preprocessing train data:   0%|          | 0/8699 [00:00<?, ?it/s]

Preprocessing val data:   0%|          | 0/2920 [00:00<?, ?it/s]

Accuracy: 0.7407534246575342
Recall (macro): 0.5083124843697095
Precision (macro): 0.5064525354781603
F1 Score (macro): 0.5050708939976601
Confusion Matrix:
[[2089  448]
 [ 309   74]]
0.5050708939976601
Directory 'gdrive/MyDrive/./CE807-24-SP/Assignment/model/2314555/Model_Gen' already exists.
Saved model to  gdrive/MyDrive/./CE807-24-SP/Assignment/model/2314555/Model_Gen/model.sav
Created a zip file gdrive/MyDrive/./CE807-24-SP/Assignment/model/2314555/Model_Gen.zip


KeyboardInterrupt: 

In [66]:
model_dis_url = 'https://drive.google.com/file/d/1--sJNJMLFvAz77HjiB8is8DDHOfY0FBG/view?usp=sharing'
test_dis(test_file, MODEL_PATH, model_dis_url)


 Start by downloading model
File downloaded successfully! gdrive/MyDrive/./CE807-24-SP/Assignment/model/2314555/test.zip
Model downloaded to gdrive/MyDrive/./CE807-24-SP/Assignment/model/2314555/test.zip
Extracted a zip file to gdrive/MyDrive/./CE807-24-SP/Assignment/model/2314555/test/

 Model is downloaded to  gdrive/MyDrive/./CE807-24-SP/Assignment/model/2314555/test/

 Data is loaded from gdrive/MyDrive/./CE807-24-SP/Assignment/data/5/test.csv
Loaded model from  gdrive/MyDrive/./CE807-24-SP/Assignment/model/2314555/test/Model_Dis/model.sav

 Output is save in  gdrive/MyDrive/./CE807-24-SP/Assignment/data/5/test.csv


'gdrive/MyDrive/./CE807-24-SP/Assignment/data/5/test.csv'

##Other Method/model End